### **2 .For a given set of training data examples stored in a .CSV file implement and demonstrate the Candidate-Elimination algorithm to output a description of the set of all hypotheses consistent with the training examples.**

### **Candidate Elimination Algorithm:**
The Candidate Elimination Algorithm is a method for learning a concept from a set of positive and negative examples. It incrementally refines the hypothesis space by generalizing and specializing the hypotheses to arrive at the most specific and general hypotheses consistent with the examples.

The candidate elimination algorithm does this by updating the general and specific boundary for each new example.

* You can consider this as an extended form of the Find-S algorithm.
* Consider both positive and negative examples.
* Actually, positive examples are used here as the Find-S algorithm (Basically they are generalizing from the specification).
* While the negative example is specified in the generalizing form.

**Terms Used:**

* **Concept learning:** Concept learning is basically the learning task of the machine (Learn by Train data)

* **General Hypothesis:** Not Specifying features to learn the machine.

* G = {‘?’, ‘?’,’?’,’?’…}: Number of attributes

* **Specific Hypothesis:** Specifying features to learn machine (Specific feature)

* **S= {‘pi’,’pi’,’pi’…}:** The number of pi depends on a number of attributes.

* **Version Space**: It is an intermediate of general hypothesis and Specific hypothesis. It not only just writes one hypothesis but a set of all possible hypotheses based on training data-set.

In [4]:
import csv

def g_0(n):
    return ("?",) * n

def s_0(n):
    return ('0',) * n

def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)


In [5]:
def fulfills(example, hypothesis):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [6]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [7]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]


In [8]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i = 0
    print("\n G[{0}]:".format(i), G)
    print("\n S[{0}]:".format(i), S)
    for xcx in examples:
        i = i + 1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx == 'Y':  # x is positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else:  # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i), G)
        print("\n S[{0}]:".format(i), S)
    print("\nFinal General Hypothesis (G):", G)
    print("\nFinal Specific Hypothesis (S):", S)
    return

In [9]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            # Keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g, h) for g in G])])
            # Remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h != h1])])
    return S



In [10]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            # Keep only specializations that have a counterpart in S
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            # Remove hypotheses less general than any other in G
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
    return G




In [13]:
# Load the examples from a CSV file
import csv
with open('/content/Lab2.csv') as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]

# Examples for testing (uncomment and use instead of loading from a file)
# examples = [
#     ('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'Y'),
#     ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'Y'),
#     ('rainy', 'cold', 'high', 'strong', 'warm', 'change', 'N'),
#     ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'Y')
# ]

# Run the candidate elimination algorithm
candidate_elimination(examples)


 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0', '0')}

 G[1]: {('?', '?', '?', '?', '?', '?')}

 S[1]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}

 G[2]: {('?', '?', '?', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[3]: {('?', '?', '?', '?', '?', 'Same'), ('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[4]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}

Final General Hypothesis (G): {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

Final Specific Hypothesis (S): {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
